In [ ]:
from grid.robot.aerial.airgen_drone import AirGenDrone 
airgen_drone__0 = AirGenDrone()

In [ ]:
# Take off
airgen_drone_0.client.takeoffAsync().join()

In [ ]:
# Get navigation mesh extents from the simulation world
nav_mesh_info = airgen_drone_0.client.getNavMeshInfo()
print("Nav mesh info: {}".format(nav_mesh_info))

In [ ]:
import airgen
import numpy as np

def sample_random_pose(nav_mesh_info):
    # Calculate bounds of the nav mesh from the given info
    amplitude = (
        np.absolute(
            np.array(
                [
                    nav_mesh_info[2]["x_val"] - nav_mesh_info[1]["x_val"],
                    nav_mesh_info[2]["y_val"] - nav_mesh_info[1]["y_val"],
                    nav_mesh_info[2]["z_val"] - nav_mesh_info[1]["z_val"],
                ]
            )
        )
        / 2.0
    )

    # Sample a random point on the nav mesh
    random_point = airgen.Vector3r(
        np.random.uniform(
            nav_mesh_info[0]["x_val"] - amplitude[0],
            nav_mesh_info[0]["x_val"] + amplitude[0],
        ),
        np.random.uniform(
            nav_mesh_info[0]["y_val"] - amplitude[1],
            nav_mesh_info[0]["y_val"] + amplitude[1],
        ),
        np.random.uniform(
            nav_mesh_info[0]["z_val"] - amplitude[2],
            nav_mesh_info[0]["z_val"] + amplitude[2],
        ),
    )

    # Sample a random yaw angle
    random_yaw = np.random.uniform(-np.pi, np.pi)

    # Return the sampled pose
    return airgen.Pose(
        random_point, airgen.Quaternionr(airgen.Vector3r(0, 0, random_yaw))
    )


In [ ]:
# Get current pose of the drone in 6-DOF
start_pose = airgen_drone_0.client.simGetVehiclePose()

# Sample a random valid pose in the environment
goal_pose = sample_random_pose(nav_mesh_info)

# Set altitude to same as the start
goal_pose.position.z_val = start_pose.position.z_val

# Compute a collision-free path between start and goal
path = airgen_drone_0.client.simPlanPath(
    start_pose.position, goal_pose.position, smooth_path=True, draw_path=True
)

In [ ]:
points = []

# Convert path into AirGen waypoints for control
for waypoint in path:
    points.append(
        airgen.Vector3r(waypoint["x_val"], waypoint["y_val"], waypoint["z_val"])
    )

# Move the drone along the planned path at a velocity of 5 m/s
velocity = 5.0
airgen_drone_0.client.moveOnPathAsync(
    points,
    velocity,
    120,
    airgen.DrivetrainType.ForwardOnly,
    airgen.YawMode(False, 0),
    -1,
    0,
).join()